In [1]:
##

## DTW를 이용

##
import csv
import random
import operator
from math import *
import numpy as np
import sys
from pyts.datasets import load_gunpoint


def DTW(A, B, window=sys.maxsize, d=lambda x, y: abs(x - y)):

    A, B = np.array(A), np.array(B)
    M, N = len(A), len(B)
    cost = sys.maxsize * np.ones((M, N))


    cost[0, 0] = d(A[0], B[0])
    for i in range(1, M):
        cost[i, 0] = cost[i - 1, 0] + d(A[i], B[0])

    for j in range(1, N):
        cost[0, j] = cost[0, j - 1] + d(A[0], B[j])
    # 나머지 행렬 채우기
    for i in range(1, M):
        for j in range(max(1, i - window), min(N, i + window)):
            choices = cost[i - 1, j - 1], cost[i, j - 1], cost[i - 1, j]
            cost[i, j] = min(choices) + d(A[i], B[j])

    n, m = N - 1, M - 1
    path = []

    while (m, n) != (0, 0):
        path.append((m, n))
        m, n = min((m - 1, n - 1), (m - 1, n), (m, n - 1), key=lambda x: cost[x[0], x[1]])

    path.append((0, 0))
    return cost[-1, -1]



# 테스트 인스턴스와 트레이닝셋들과의 거리를 구한다. 그리고, 트레이닝 레이블에 거리를 붙여서 한다.
# DTW 이용해서 거리 구한다
def getNeighbors(trainingSet, testInstance, trainlabel, testlabel, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = DTW(testInstance, trainingSet[x], window = 23)
        distances.append((trainlabel[x], dist))
 
    distances.sort(key=operator.itemgetter(1))

    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])

    return neighbors


def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]


def getAccuracy(y_test, predictions):
   correct = 0
   for x in range(len(y_test)):
      if y_test[x] == predictions[x]:
         correct += 1
   return (correct/float(len(y_test))) * 100.0
   
def main():

    trainingSet=[]
    testSet=[]
    split = 0.67
    X_train, X_test, y_train, y_test = load_gunpoint(return_X_y=True)
    print ('Train set: ' + repr(len(X_train)))
    print ('Test set: ' + repr(len(X_test)))
    
    predictions=[]
    k = 1
    
    for x in range(len(X_test)):
        neighbors = getNeighbors(X_train, X_test[x], y_train, y_test,  k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('>' + str(x) + 'th predicted=' + repr(result) + ', actual=' + repr(y_test[x]))
        
    accuracy = getAccuracy(y_test, predictions)
    print('DTW: Accuracy: ' + repr(accuracy) + '%')

main()

NameError: name 'fetch_ucr_dataset' is not defined